### comment out the code below to install required packages

In [ ]:
# code to install all the packages we need
# %pip install numpy
# %pip install pandas
# %pip install yfinance
# %pip install requests
# %pip install beautifulsoup4
# %pip install scikit-learn
# %pip install TA-lib
# %pip install joblib
# %pip install pickle
# %pip install Riskfolio-lib


In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import *
from ibapi.order import *
import ibapi

import requests
import bs4 as bs
import datetime
import time
import threading
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV  
import joblib

import talib as ta
from talib import MA_Type
import pickle


In [2]:
init_event = threading.Event() # init event
id_event = threading.Event()
mkt_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [3]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        self.mkt_price = []
        # create a connection with IBKR
        self.connect(address, port, cid)
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.value = 0
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib: ibapi.common.TickAttrib):
            print("Tick Price. Ticker Id:", reqId, "tickType:", tickType, "Price:", price)
            if tickType == 9: # if tickType is Close Price
                self.mkt_price.append([reqId, price])
                mkt_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.concat([self.last_portfolio,
                                        pd.DataFrame([[contract.symbol, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL]],
                                                    columns=self.last_portfolio.columns)],
                                        ignore_index=True)
        port_event.set()

    # query total value
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        self.value = float(value)
        print("Total value of the account: ", self.value)
    
    def accountSummaryEnd(self, reqId: int):
        print('end of account summary')
        value_event.set()

In [4]:
# Connect to the TWS API
app = App('127.0.0.1', 7497, 1000)
init_event.wait() # wait until it's connected
init_event.clear()

ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR:ibapi.wrapper:ERROR -1 2158 Sec-def data farm connection is OK:secdefil


NextValidId: 33


In [5]:
app.reqMarketDataType(1) # in case it doesn't work, change 1 to 3

In [6]:
with open('portfolio_list.pickle', 'rb') as f:
    portfolio_list = pickle.load(f)

print(portfolio_list)

['ADP', 'AJG', 'KMX', 'DVN', 'FITB', 'LNC', 'LYV', 'TRGP', 'YUM', 'ZBH']


# query the current portfolio


In [7]:
app.reqAccountUpdates(True, "DU6730183") #TODO: replace the DU222526 with our account ID
# port_event.wait()
# port_event.clear()

time.sleep(10)
last_portfolio = app.last_portfolio
app.reqAccountUpdates(False, "DU6730183") 

ERROR:ibapi.wrapper:ERROR -1 2100 API client has been unsubscribed from account data.


In [8]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,ADP,382,223.554230,85397.72,221.323115,852.29,0.0
1,AJG,457,187.752289,85802.80,184.931236,1289.22,0.0
2,DVN,1591,56.326000,89614.67,54.786188,2449.84,0.0
3,FITB,2373,35.976097,85371.28,35.805000,406.01,0.0
4,KMX,1227,69.712234,85536.91,68.310110,1720.41,0.0
5,LNC,2638,30.960720,81674.38,31.723863,-2013.17,0.0
6,LYV,1130,72.837921,82306.85,75.505000,-3013.80,0.0
7,TRGP,1128,78.200516,88210.18,76.403449,2027.09,0.0
8,YUM,659,128.827606,84897.39,128.620675,136.37,0.0
9,ZBH,689,124.998802,86124.17,122.485000,1732.01,0.0


In [9]:
app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
app.reqAccountSummary(9001, "All", "NetLiquidation")
value_event.wait()
value_event.clear()
# time.sleep(3)
total_value = app.value
app.cancelAccountSummary(9001)

Total value of the account:  952825.65
end of account summary


In [10]:
total_value

952825.65

# implementation
* decide what to buy & sell, as well as the quantity

In [11]:

# get the market price
for i, t in enumerate(portfolio_list):
    contract = Contract()
    contract.symbol = t
    contract.secType = "STK"
    contract.currency = "USD"
    contract.exchange = "SMART"
    app.reqMktData(i, contract, "", False, False, [])
    mkt_event.wait()
    mkt_event.clear()
    app.cancelMktData(i)

mkt_df = pd.DataFrame(app.mkt_price, columns=['reqId', 'mkt_price'])
assert sum(mkt_df.isnull().any()) == 0, "didn't get all the market price"
assert len(mkt_df) == len(portfolio_list), "didn't get all the market price"


Tick Price. Ticker Id: 0 tickType: 1 Price: 223.16
Tick Price. Ticker Id: 0 tickType: 2 Price: 223.35
Tick Price. Ticker Id: 0 tickType: 4 Price: 223.63
Tick Price. Ticker Id: 0 tickType: 6 Price: 224.16
Tick Price. Ticker Id: 0 tickType: 7 Price: 222.8
Tick Price. Ticker Id: 0 tickType: 9 Price: 222.55
Tick Price. Ticker Id: 0 tickType: 14 Price: 223.13
Tick Price. Ticker Id: 1 tickType: 1 Price: 187.61
Tick Price. Ticker Id: 1 tickType: 2 Price: 187.74
Tick Price. Ticker Id: 1 tickType: 4 Price: 187.43
Tick Price. Ticker Id: 1 tickType: 6 Price: 187.74
Tick Price. Ticker Id: 1 tickType: 7 Price: 186.53
Tick Price. Ticker Id: 1 tickType: 9 Price: 187.02
Tick Price. Ticker Id: 1 tickType: 14 Price: 187.05
Tick Price. Ticker Id: 2 tickType: 1 Price: 69.56
Tick Price. Ticker Id: 2 tickType: 2 Price: 69.68
Tick Price. Ticker Id: 2 tickType: 4 Price: 69.74
Tick Price. Ticker Id: 2 tickType: 6 Price: 70.07
Tick Price. Ticker Id: 2 tickType: 7 Price: 67.5
Tick Price. Ticker Id: 2 tickType: 9

In [12]:
target_portfolio = pd.DataFrame(columns=['tickers', 'weight', 'mkt_price'])
target_portfolio['tickers'] = portfolio_list
target_portfolio['weight'] = np.repeat(1/len(portfolio_list), len(portfolio_list))
target_portfolio['mkt_price'] = mkt_df['mkt_price']

In [13]:
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value * 0.95) / target_portfolio['mkt_price'])

In [14]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,ADP,382,223.554230,85397.72,221.323115,852.29,0.0
1,AJG,457,187.752289,85802.80,184.931236,1289.22,0.0
2,DVN,1591,56.326000,89614.67,54.786188,2449.84,0.0
3,FITB,2373,35.976097,85371.28,35.805000,406.01,0.0
4,KMX,1227,69.712234,85536.91,68.310110,1720.41,0.0
5,LNC,2638,30.960720,81674.38,31.723863,-2013.17,0.0
6,LYV,1130,72.837921,82306.85,75.505000,-3013.80,0.0
7,TRGP,1128,78.200516,88210.18,76.403449,2027.09,0.0
8,YUM,659,128.827606,84897.39,128.620675,136.37,0.0
9,ZBH,689,124.998802,86124.17,122.485000,1732.01,0.0


In [15]:
target_portfolio

,tickers,weight,mkt_price,quantity
0,ADP,0.1,222.55,385.0
1,AJG,0.1,187.02,458.0
2,KMX,0.1,67.86,1263.0
3,DVN,0.1,55.70,1539.0
4,FITB,0.1,35.62,2407.0
5,LNC,0.1,30.68,2795.0
6,LYV,0.1,72.46,1183.0
7,TRGP,0.1,77.25,1110.0
8,YUM,0.1,128.68,666.0
9,ZBH,0.1,122.99,697.0


# figure out the best order of our order. (To save commission fee)

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [20]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []
# Create a list to store the stocks that need to buy
stocks_to_buy = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['tickers']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = float(last_portfolio_stock['quantity'].values[0])
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append([row['tickers'], quantity_diff])

        if row['quantity'] > last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = row['quantity'] - last_portfolio_quantity 
            # Add the stock to the list of stocks to buy
            stocks_to_buy.append([row['tickers'], quantity_diff])

# we also need to sell the stocks that are in last_portfolio but not in target_portfolio
# Find the difference between the two portfolios
new_tickers_to_sell = set(last_portfolio['ticker']).difference(set(target_portfolio['tickers']))

for ticker in new_tickers_to_sell:
    # Find the quantity of the stock to sell from the last_portfolio
    quantity = last_portfolio.loc[last_portfolio['ticker'] == ticker, 'quantity'].values[0]
    # append to stocks to sell
    if quantity != 0:
        stocks_to_sell.append([ticker, quantity])

# we also need to buy the stocks that are in target_protfolio but not in last_portfolio
# Find the difference between the two portfolios
new_tickers_to_buy = set(target_portfolio['tickers']).difference(set(last_portfolio['ticker']))

for ticker in new_tickers_to_buy:
    # Find the quantity of the stock to buy from the target_portfolio
    quantity = target_portfolio.loc[target_portfolio['tickers'] == ticker, 'quantity'].values[0]
    # append to stocks to buy
    if quantity != 0:
        stocks_to_buy.append([ticker, quantity])

In [21]:
stocks_to_buy

[['ADP', 3.0],
 ['AJG', 1.0],
 ['KMX', 36.0],
 ['FITB', 34.0],
 ['LNC', 157.0],
 ['LYV', 53.0],
 ['YUM', 7.0],
 ['ZBH', 8.0]]

In [22]:
stocks_to_sell

[['DVN', 52.0], ['TRGP', 18.0]]

# place order

In [23]:
app.reqIds(-1) # require a new id
id_event.wait()
id_event.clear()
orderid = app.nextValidOrderId
for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1

    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

count = 0
for stock in stocks_to_buy:
    # Replace the place holder values with the appropriate values
    app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
    app.reqAccountSummary(9001, "All", "SettledCash")
    value_event.wait()
    value_event.clear()
    cash_value = app.value
    if count >= len(stocks_to_buy)/2 and cash_value <= 0:
        break # stop buying when cash is below 0
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "BUY"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1
    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()
    count +=1

NextValidId: 33
openOrder. orderId:34, contract:754442,DVN,STK,,0,?,,SMART,,USD,DVN,DVN,False,,,,combo:, order:34,1000,54388340: MKT SELL 52@56.26 GTC
orderStatus. orderId: 34, status: Submitted, filled: 0, remaining:52, avgFillPrice: 0.0, permId:54388340, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:35, contract:81671838,TRGP,STK,,0,?,,SMART,,USD,TRGP,TRGP,False,,,,combo:, order:35,1000,54388341: MKT SELL 18@78.13 GTC
orderStatus. orderId: 35, status: Submitted, filled: 0, remaining:18, avgFillPrice: 0.0, permId:54388341, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
end of account summary
openOrder. orderId:36, contract:4661,ADP,STK,,0,?,,SMART,,USD,ADP,NMS,False,,,,combo:, order:36,1000,54388342: MKT BUY 3@223.24 GTC
orderStatus. orderId: 36, status: Submitted, filled: 0, remaining:3, avgFillPrice: 0.0, permId:54388342, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
execDetails.

openOrder. orderId:43, contract:12851783,ZBH,STK,,0,?,,SMART,,USD,ZBH,ZBH,False,,,,combo:, order:43,1000,54388349: MKT BUY 8@125.15 GTC
orderStatus. orderId: 43, status: Submitted, filled: 0, remaining:8, avgFillPrice: 0.0, permId:54388349, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:43, contract:12851783,ZBH,STK,,0,?,,SMART,,USD,ZBH,ZBH,False,,,,combo:, order:43,1000,54388349: MKT BUY 8@0 GTC
orderStatus. orderId: 43, status: Submitted, filled: 0, remaining:8, avgFillPrice: 0.0, permId:54388349, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:43, contract:12851783,ZBH,STK,,0,?,,SMART,,USD,ZBH,ZBH,False,,,,combo:, order:43,1000,54388349: MKT BUY 8@0 GTC
orderStatus. orderId: 43, status: Submitted, filled: 0, remaining:8, avgFillPrice: 0.0, permId:54388349, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
execDetails. reqId: -1, contract: 12851783,ZBH,STK,,0,,,ISLAND

In [24]:
app.disconnect()